In [3]:
import os
import numpy as np
import pydicom
import scipy
import cv2
import matplotlib.pyplot as plt

In [15]:
file_path = "C:/Users/clarynse/nnUNetFrame/dataset/nnUNet_raw/Dataset002_ThyroidSeg/imagesTr"

In [17]:
def load_dicom(file_path):
    dicom_data = pydicom.dcmread(file_path)
    image_data = dicom_data.pixel_array  #Get the pixel array
    return image_data
    

In [25]:
print("\nFiles in imagesTr:", os.listdir(file_path)[:10])  # Print first 10 files


Files in imagesTr: ['.ipynb_checkpoints', 'case001_0000.nrrd', 'case002_0000.nrrd', 'case003_0000.nrrd', 'case004_0000.nrrd', 'case005_0000.nrrd', 'case006_0000.nrrd', 'case007_0000.nrrd', 'case008_0000.nrrd', 'case009_0000.nrrd']


In [29]:
print("Dataset Path Exists:", os.path.exists(file_path))

Dataset Path Exists: True


In [ ]:
#Example usage
dicom_image = load_dicom("path_to_dicom_file.dcm")

In [ ]:
#Normalize the image

def normalize_image(image_data):
    #Normalize to the range [0,1]
    image_normalized = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
    return image_normalized

In [ ]:
#Example usage
dicom_image_normalized = normalize_image(dicom_image)

In [ ]:
#Rescaling the image

def resize_image(image_data, target_size=(128,128)):
    resized_image = cv2.resize(image_data, target_size, interpolation=cv2.INTER_LINEAR)
    return resized_image

In [ ]:
#Example usage
resized_image = resize_image(dicom_image_normalized, target_size=(128,128))

In [ ]:
#Slice Selection
#For manual selection select the desired slice:
#Load the DICOM series(assuming they are in a folder)
def load_dicom_series(directory_path):
    dicom_files = sorted(f for f in os.listdir(directory_path) if f.endswith('.dcm')])
    dicom_images = [load_dicom(os.path.join(directory_path,f)) for f in dicom_files]
    return np.array(dicom_images)
    

In [ ]:
#Example usage
dicom_series = load_dicom_series("path_to_directory_with_dicom_images")


In [ ]:
#Select Specific Slice (eg.50th slice)
slice_index = 50
selected_slice = dicom_series[slice_index]


In [ ]:
#Apply thresholding for region of interest(RoI)
def apply_threshold(image_data, lower_threshold, upper_threshold):
    thresholded_image = np.clip(image_data, lower_threshold, upper_threshold)
    return thresholded_image

In [ ]:
#Example usage
thresholded_image = apply_threshold(resized_image, lower_threshold=0, upper_threshold=500)


In [ ]:
#Apply smoothing (optional)
def apply_smoothing(image_data, kernel_size=(3,3)):
    smoothed_image = cv2.GaussianBlur(image_data, kernel_size, 0)
    return smoothed_image
    

In [ ]:
#Example usage
smoothed-image = apply_smoothing(thresholded_image)


In [ ]:
#Data Augmentation(Optional)
#For Deep Learning tasks, augmenting the dataset can help improve model generalization. Transformations like rotation, flipping, scaling, or random crops to your images.
def augment_image(image_data):
    #Example augmentation:random flip
    if np.random.rand() > 0.5:
        image_data = np.flipud(image_data) #Flip vertically
    return image_data

In [ ]:
#Example usage
augmented_image = augment_image(smoothed_image)


In [ ]:
#Save the preprocessed images
def save_image(image_data, save_path):
    plt.imsave(save_path, image_data, cmap='gray')

In [ ]:
#Example usage
save_image(augmented_image, "preprocessed_image.png")

In [ ]:
#Organizing and labelling
#If you are working with a set of images(like a whole scan with multiple slices), make sure to organize your data and create proper labels if necessary for supervised learning tasks. You may also want to store additional metadata, like slice position or other relevant information from the DICOM headers.